In [4]:
# Load the necessary libraries
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import  classification_report
import seaborn as sns
from PIL import Image
import timm

In [5]:
# Load the dataset from the CSV file
df = pd.read_csv('labels.csv')

# Drop the unnecessary column 'Unnamed: 0' from the DataFrame
df = df.drop('Unnamed: 0', axis=1)

# Convert the image paths to string type
df['pth'] = df['pth'].astype(str)

# Define the label mapping for emotions to numerical values
classes = {
    "anger": 0,
    "disgust": 1,
    "fear": 2,
    "happy": 3,
    "sad": 4,
    "surprise": 5,
    "netural": 6
}
num_classes = len(classes) # Number of classes

# Drop rows with labels that are not in the defined classes
df = df[df['label'].isin(classes.keys())]

# Apply the label mapping to the 'label' column
df['label'] = df['label'].map(classes)

# Split the dataset into training and validation sets with stratification
train_df, validation_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

In [6]:
# Define the transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize the images
])


# Create a dataset using the DataFrame without a custom class
class CustomImageDataset(Dataset):
    def __init__(self, df, root_dir='./', transform=None):
        self.df = df # DataFrame containing the image paths and labels
        self.root_dir = root_dir # Root directory containing the images
        self.transform = transform # Transformations to apply to the images
        self.image_paths = self.df['pth'].values # Image paths
        self.labels = self.df['label'].values # Corresponding labels

    def __len__(self):
        """Return the total number of samples in the dataset."""
        return len(self.image_paths)

    def __getitem__(self, idx):
        """Retrieve a sample from the dataset."""
        # Get the image path
        image_path = os.path.join(self.root_dir, self.image_paths[idx])
        
        # Open and convert the image to RGB
        image = Image.open(image_path).convert('RGB')
        
        # Apply the transform if it exists
        if self.transform:
            image = self.transform(image)
        
        # Get the class index from the label
        class_idx = self.labels[idx]
        
        # Create a one-hot encoded label tensor
        label = torch.zeros(num_classes)
        label[class_idx] = 1
        
        return image, label # Return the image and one-hot encoded label


# Create datasets for training and validation
train_dataset = CustomImageDataset(train_df, transform=transform)
validation_dataset = CustomImageDataset(validation_df, transform=transform)

# Get number of images in each dataset
num_train_images = len(train_dataset)
num_validation_images = len(validation_dataset)

print(f'Number of training images: {num_train_images}')
print(f'Number of validation images: {num_validation_images}')

# Create DataLoaders for training and validation
batch_size = 10
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

Number of training images: 17656
Number of validation images: 4414


In [10]:
# Define a Trainer class to handle the training and validation process
class Trainer:
    def __init__(self, save_path, num_epochs):
        self.num_epochs = num_epochs # Number of training epochs
        self.save_path = save_path # Path to save model checkpoints

        # Initialize the model
        self.model = timm.create_model('convnext_pico.d1_in1k', pretrained=True)
        self.model.head.fc = nn.Linear(512, 7) # Change the number of output classes to 7

        # Define loss function (cross-entropy loss)
        self.criterion = nn.CrossEntropyLoss()

        # Define optimizer (Adam optimizer)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.00001)

        # Set device (GPU if available, else CPU)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device) # Move the model to the device
        print(f"Using device: {self.device} \n")

    def validate(self, epoch):
        """Perform validation on the validation set."""
        self.model.eval() # Set the model to evaluation mode
        print("Validating...")

        running_loss = 0.0
        total_loss = 0.0
        total = 0
        correct = 0
        with torch.no_grad(): # Disable gradient computation
            for i, data in enumerate(validation_loader):
                images, labels = (data[0].to(self.device), data[1].to(self.device))
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)

                # Calculate loss
                total_loss += loss.item()
                running_loss += loss.item()

                # Calculate accuracy
                predicted = torch.argmax(outputs.data, 1)
                predicted_labels = torch.argmax(labels.data, 1)
                total += labels.size(0)
                correct += (predicted == predicted_labels).sum().item()

                # Calculate average validation loss for the last 100 batches
                if (i + 1) % 100 == 0:
                    print('Validation Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, (i + 1) * len(images), len(validation_loader.dataset),
                        100. * (i + 1) / len(validation_loader), running_loss / 100))
                    running_loss = 0.

        avg_loss = total_loss / len(validation_loader) # Average validation loss
        accuracy = 100 * correct / total # Validation accuracy

        # Save the model checkpoint
        os.makedirs(self.save_path, exist_ok=True)
        torch.save(self.model, self.save_path + os.sep + 'cnn-{}.pt'.format(epoch))

        return avg_loss, accuracy # Return average validation loss and accuracy
    
    def train_one_epoch(self, epoch):
        """Train the model for one epoch."""
        self.model.train() # Set the model to training mode
        print("Training...")

        running_loss = 0.0
        total_loss = 0.0
        total = 0
        correct = 0
        for i, data in enumerate(train_loader):
            images, labels = (data[0].to(self.device), data[1].to(self.device))
            self.optimizer.zero_grad() # Clear the gradients
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)
            
            loss.backward() # Backpropagate the loss
            self.optimizer.step() # Update the model parameters
            
            # Calculate loss
            total_loss += loss.item()
            running_loss += loss.item()

            # Calculate accuracy
            predicted = torch.argmax(outputs.data, 1)
            predicted_labels = torch.argmax(labels.data, 1)
            total += labels.size(0)
            correct += (predicted == predicted_labels).sum().item()

            # Calculate average training loss for the last 100 batches
            if (i + 1) % 100 == 0:
                print('Training Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, (i + 1) * len(images), len(train_loader.dataset),
                    100. * (i + 1) / len(train_loader), running_loss / 100))
                running_loss = 0.

        avg_loss = total_loss / len(train_loader) # Average training loss
        accuracy = 100 * correct / total # Training accuracy

        return avg_loss, accuracy # Return average training loss and accuracy
    
    def train(self):
        self.train_losses = [] # Store training losses
        self.validation_losses = [] # Store validation losses
        self.train_accuracies = [] # Store training accuracies
        self.validation_accuracies = [] # Store validation accuracies

        for epoch in range(self.num_epochs):
            print('EPOCH {}:'.format(epoch+1))

            # Train and validate for one epoch
            train_loss, train_accuracy = self.train_one_epoch(epoch+1)
            validation_loss, validation_accuracy = self.validate(epoch+1)
            print('Training Loss: {:.8f} \tValidation Loss {:.8f} \tTraining Accuracy {:.3f}% \tValidation Accuracy {:.3f}% \n'
                                                        .format(train_loss,validation_loss, train_accuracy, validation_accuracy))
            
            # Store training and validation metrics
            self.train_losses.append(train_loss.cpu().item() if torch.is_tensor(train_loss) else train_loss)
            self.train_accuracies.append(train_accuracy.cpu().item() if torch.is_tensor(train_accuracy) else train_accuracy)
            self.validation_losses.append(validation_loss.cpu().item() if torch.is_tensor(validation_loss) else validation_loss)
            self.validation_accuracies.append(validation_accuracy.cpu().item() if torch.is_tensor(validation_accuracy) else validation_accuracy)
            

# Set the path for saving model checkpoints and number of epochs
save_path = 'model_checkpoints'
num_epochs = 40
trainer = Trainer(save_path, num_epochs) # Instantiate the Trainer class
trainer.train() # Start the training process



Using device: cpu 

EPOCH 1:
Training...
Training Epoch: 1 [1000/17656 (6%)]	Loss: 1.788902
Training Epoch: 1 [2000/17656 (11%)]	Loss: 1.562347
Training Epoch: 1 [3000/17656 (17%)]	Loss: 1.428034
Training Epoch: 1 [4000/17656 (23%)]	Loss: 1.286786
Training Epoch: 1 [5000/17656 (28%)]	Loss: 1.230498
Training Epoch: 1 [6000/17656 (34%)]	Loss: 1.178919
Training Epoch: 1 [7000/17656 (40%)]	Loss: 1.144751
Training Epoch: 1 [8000/17656 (45%)]	Loss: 1.111628
Training Epoch: 1 [9000/17656 (51%)]	Loss: 1.068705
Training Epoch: 1 [10000/17656 (57%)]	Loss: 1.038430
Training Epoch: 1 [11000/17656 (62%)]	Loss: 1.013895
Training Epoch: 1 [12000/17656 (68%)]	Loss: 1.004499
Training Epoch: 1 [13000/17656 (74%)]	Loss: 0.937379
Training Epoch: 1 [14000/17656 (79%)]	Loss: 0.969765
Training Epoch: 1 [15000/17656 (85%)]	Loss: 0.924308
Training Epoch: 1 [16000/17656 (91%)]	Loss: 0.904880
Training Epoch: 1 [17000/17656 (96%)]	Loss: 0.875361
Validating...
Validation Epoch: 1 [1000/4414 (23%)]	Loss: 0.818413
V